In [31]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
import ipywidgets as widgets
from IPython.display import display

In [2]:
data = pd.read_csv('../../MA_PREDICTOR/data/ma_detailed_data_car_clean.csv')

In [3]:
data.drop('month', inplace=True, axis=1)

In [4]:
horizons = [1, 3, 5, 10]
for horizon in horizons:
    data = data[(abs(data[f'car_{horizon}']) <= 0.3)]

In [5]:
data

,announcement_date,target_status,acquisition_count,shares_at_announcement,shares_acquired,consideration_offered,bidder_count,rel_deal_value,cross_border,relatedness,...,business_sector_ac,economic_sector_target,business_sector_target,cluster_category,a_fin_adv_count,t_fin_adv_count,car_1,car_3,car_5,car_10
0,04/01/2005,public,23,no,full,Cash,1,0.024922,cross_border,not_related,...,Food & Drug Retailing,Industrials,Transportation,divestiture,1,0,-0.009362,0.001308,0.018041,0.056428
1,11/01/2005,public,18,no,full,Other,1,0.008012,cross_border,industry,...,Banking & Investment Services,Financials,Banking & Investment Services,asset_driven_op,0,0,-0.011249,0.004454,0.012696,0.044807
2,12/01/2005,public,8,no,not_full,Cash,1,0.382792,national,industry,...,Mineral Resources,Basic Materials,Mineral Resources,divestiture,0,0,0.013061,0.056958,0.055917,0.016918
3,18/01/2005,public,40,no,full,Other,1,0.014665,cross_border,industry,...,Banking & Investment Services,Financials,Banking & Investment Services,asset_driven_op,0,0,0.060476,-0.010042,0.002116,0.040544
4,26/01/2005,public,0,no,full,Other,1,2.000163,cross_border,industry,...,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,asset_driven_op,0,0,-0.093760,-0.082288,-0.082288,-0.061740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,16/07/2021,others,9,no,full,Cash,1,0.004449,cross_border,not_related,...,Utilities,Energy,Energy - Fossil Fuels,divestiture,0,1,-0.015629,-0.045382,-0.025400,-0.054948
3021,19/07/2021,others,1,no,full,Other,1,0.104705,cross_border,industry,...,Software & IT Services,Technology,Software & IT Services,asset_driven_op,0,0,0.088395,0.122123,0.059973,0.066900
3022,19/07/2021,others,6,no,full,Other,1,0.270094,cross_border,industry,...,Software & IT Services,Technology,Software & IT Services,divestiture,0,0,0.068755,0.110112,0.102726,0.120972
3023,28/07/2021,others,21,no,full,Cash,1,0.232595,cross_border,not_related,...,Applied Resources,Consumer Cyclicals,Cyclical Consumer Services,divestiture,1,0,0.022186,0.041487,0.041487,0.016503


In [6]:
data['car_class_1'] = data['car_1']

In [7]:
data['car_class_3'] = data['car_3']

In [8]:
data['car_class_5'] = data['car_5']

In [9]:
data['car_class_10'] = data['car_10']

In [145]:
data.columns

Index(['announcement_date', 'target_status', 'acquisition_count',
       'shares_at_announcement', 'shares_acquired', 'consideration_offered',
       'bidder_count', 'rel_deal_value', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
       'business_sector_target', 'cluster_category', 'a_fin_adv_count',
       't_fin_adv_count', 'car_1', 'car_3', 'car_5', 'car_10', 'car_class_1',
       'car_class_3', 'car_class_5', 'car_class_10'],
      dtype='object')

In [11]:
def trans_class(x):
    if x > 0.01:
        return "positive"
    elif x < -0.01:
        return "negative"
    else:
        return "neutral"

In [12]:
horizons = [1, 3, 5, 10]
for horizon in horizons:
    data[f'car_class_{horizon}']=data[f'car_class_{horizon}'].apply(trans_class)

In [13]:
data

,announcement_date,target_status,acquisition_count,shares_at_announcement,shares_acquired,consideration_offered,bidder_count,rel_deal_value,cross_border,relatedness,...,a_fin_adv_count,t_fin_adv_count,car_1,car_3,car_5,car_10,car_class_1,car_class_3,car_class_5,car_class_10
0,04/01/2005,public,23,no,full,Cash,1,0.024922,cross_border,not_related,...,1,0,-0.009362,0.001308,0.018041,0.056428,neutral,neutral,positive,positive
1,11/01/2005,public,18,no,full,Other,1,0.008012,cross_border,industry,...,0,0,-0.011249,0.004454,0.012696,0.044807,negative,neutral,positive,positive
2,12/01/2005,public,8,no,not_full,Cash,1,0.382792,national,industry,...,0,0,0.013061,0.056958,0.055917,0.016918,positive,positive,positive,positive
3,18/01/2005,public,40,no,full,Other,1,0.014665,cross_border,industry,...,0,0,0.060476,-0.010042,0.002116,0.040544,positive,negative,neutral,positive
4,26/01/2005,public,0,no,full,Other,1,2.000163,cross_border,industry,...,0,0,-0.093760,-0.082288,-0.082288,-0.061740,negative,negative,negative,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,16/07/2021,others,9,no,full,Cash,1,0.004449,cross_border,not_related,...,0,1,-0.015629,-0.045382,-0.025400,-0.054948,negative,negative,negative,negative
3021,19/07/2021,others,1,no,full,Other,1,0.104705,cross_border,industry,...,0,0,0.088395,0.122123,0.059973,0.066900,positive,positive,positive,positive
3022,19/07/2021,others,6,no,full,Other,1,0.270094,cross_border,industry,...,0,0,0.068755,0.110112,0.102726,0.120972,positive,positive,positive,positive
3023,28/07/2021,others,21,no,full,Cash,1,0.232595,cross_border,not_related,...,1,0,0.022186,0.041487,0.041487,0.016503,positive,positive,positive,positive


In [139]:
data['t_fin_adv_count'].unique()

array([0, 2, 1, 3, 4, 7])

# Encoding the target

In [14]:
# creating instance of labelencoder
labelencoder = LabelEncoder()


In [15]:
# Assigning numerical values and storing in another column
data['car_class_1'] = labelencoder.fit_transform(data['car_class_1'])
data['car_class_3'] = labelencoder.fit_transform(data['car_class_3'])
data['car_class_5'] = labelencoder.fit_transform(data['car_class_5'])
data['car_class_10'] = labelencoder.fit_transform(data['car_class_10'])

In [16]:
data

,announcement_date,target_status,acquisition_count,shares_at_announcement,shares_acquired,consideration_offered,bidder_count,rel_deal_value,cross_border,relatedness,...,a_fin_adv_count,t_fin_adv_count,car_1,car_3,car_5,car_10,car_class_1,car_class_3,car_class_5,car_class_10
0,04/01/2005,public,23,no,full,Cash,1,0.024922,cross_border,not_related,...,1,0,-0.009362,0.001308,0.018041,0.056428,1,1,2,2
1,11/01/2005,public,18,no,full,Other,1,0.008012,cross_border,industry,...,0,0,-0.011249,0.004454,0.012696,0.044807,0,1,2,2
2,12/01/2005,public,8,no,not_full,Cash,1,0.382792,national,industry,...,0,0,0.013061,0.056958,0.055917,0.016918,2,2,2,2
3,18/01/2005,public,40,no,full,Other,1,0.014665,cross_border,industry,...,0,0,0.060476,-0.010042,0.002116,0.040544,2,0,1,2
4,26/01/2005,public,0,no,full,Other,1,2.000163,cross_border,industry,...,0,0,-0.093760,-0.082288,-0.082288,-0.061740,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,16/07/2021,others,9,no,full,Cash,1,0.004449,cross_border,not_related,...,0,1,-0.015629,-0.045382,-0.025400,-0.054948,0,0,0,0
3021,19/07/2021,others,1,no,full,Other,1,0.104705,cross_border,industry,...,0,0,0.088395,0.122123,0.059973,0.066900,2,2,2,2
3022,19/07/2021,others,6,no,full,Other,1,0.270094,cross_border,industry,...,0,0,0.068755,0.110112,0.102726,0.120972,2,2,2,2
3023,28/07/2021,others,21,no,full,Cash,1,0.232595,cross_border,not_related,...,1,0,0.022186,0.041487,0.041487,0.016503,2,2,2,2


In [81]:
data['economic_sector_ac'].unique()

array(['Consumer Non-Cyclicals', 'Financials', 'Basic Materials',
       'Energy', 'Industrials', 'Real Estate', 'Technology', 'Healthcare',
       'Consumer Cyclicals', 'Utilities',
       'Academic & Educational Services'], dtype=object)

In [134]:
data['cluster_category'].unique()

array(['divestiture', 'asset_driven_op', 'private_companies',
       'bankruptcy', 'possible_problematic', 'Unknown'], dtype=object)

In [18]:
y_1=data['car_class_1']

In [19]:
y_3=data['car_class_3']

In [20]:
y_5=data['car_class_5']

In [21]:
y_10=data['car_class_10']

In [22]:
X=data[['target_status', 'acquisition_count',
       'shares_at_announcement', 'shares_acquired', 'consideration_offered',
       'bidder_count', 'rel_deal_value', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
       'business_sector_target', 'cluster_category', 'a_fin_adv_count',
       't_fin_adv_count']]

# Car_1

## Splitting of the dataset

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_1, test_size=0.3, random_state=0)

## Pipeline and GridSearch

In [41]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [42]:
num_transformer= MinMaxScaler()

In [43]:
cat_features=['target_status', 'shares_at_announcement', 'shares_acquired', 
              'consideration_offered',
               'cross_border', 'relatedness',
               'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
               'business_sector_target', 'cluster_category']

In [44]:
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['acquisition_count', 'rel_deal_value']),
    ('cat_transformer', cat_transformer, cat_features)], remainder='passthrough')

## Baseline

In [45]:
baseline_model_str = DummyClassifier(strategy="stratified") 

In [46]:
cross_val_score(baseline_model_str, X_train, y_train, cv=5, scoring='accuracy').mean()

0.3549942405792331

## SGDClassifier

In [47]:
model= SGDClassifier()

In [48]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)])

In [49]:
grid = {
    'model__alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], 
    'model__loss': ['hinge', 'log', 'modified_huber'],
    'model__penalty': ['l2', 'l1', 'elasticnet'],
    'model__n_jobs': [-1]
}

In [50]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = 'accuracy',
                      cv = 5,
                      n_jobs=-1) 

In [51]:
# Fit data to Grid Search
search.fit(X_train,y_train);

In [52]:
# Best score 1
search.best_score_

0.4224194268788641

# Car_3

## Splitting of the dataset

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_3, test_size=0.3, random_state=0)

## Pipeline and GridSearch

In [54]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [55]:
num_transformer= MinMaxScaler()

In [56]:
cat_features=['target_status', 'shares_at_announcement', 'shares_acquired', 
              'consideration_offered',
               'cross_border', 'relatedness',
               'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
               'business_sector_target', 'cluster_category']

In [57]:
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['acquisition_count', 'rel_deal_value']),
    ('cat_transformer', cat_transformer, cat_features)], remainder='passthrough')

## Baseline

In [58]:
baseline_model_str = DummyClassifier(strategy="stratified") 

In [59]:
cross_val_score(baseline_model_str, X_train, y_train, cv=5, scoring='accuracy').mean()

0.37149791955617195

## SGDClassifier

In [60]:
model= SGDClassifier()

In [61]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)])

In [62]:
grid = {
    'model__alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], 
    'model__loss': ['hinge', 'log', 'modified_huber'],
    'model__penalty': ['l2', 'l1', 'elasticnet'],
    'model__n_jobs': [-1]
}

In [63]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = 'accuracy',
                      cv = 5,
                      n_jobs=-1) 

In [64]:
# Fit data to Grid Search
search.fit(X_train,y_train);

In [65]:
# Best score 1
search.best_score_

0.44859658196008373

# Car_5

## Splitting of the dataset

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_5, test_size=0.3, random_state=0)

## Pipeline and GridSearch

In [67]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [68]:
num_transformer= MinMaxScaler()

In [69]:
cat_features=['target_status', 'shares_at_announcement', 'shares_acquired', 
              'consideration_offered',
               'cross_border', 'relatedness',
               'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
               'business_sector_target', 'cluster_category']

In [70]:
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['acquisition_count', 'rel_deal_value']),
    ('cat_transformer', cat_transformer, cat_features)], remainder='passthrough')

## Baseline

In [71]:
baseline_model_str = DummyClassifier(strategy="stratified") 

In [72]:
cross_val_score(baseline_model_str, X_train, y_train, cv=5, scoring='accuracy').mean()

0.37195279625755184

## SGDClassifier

In [73]:
model= SGDClassifier()

In [74]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)])

In [75]:
grid = {
    'model__alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], 
    'model__loss': ['hinge', 'log', 'modified_huber'],
    'model__penalty': ['l2', 'l1', 'elasticnet'],
    'model__n_jobs': [-1]
}

In [76]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = 'accuracy',
                      cv = 5,
                      n_jobs=-1) 

In [77]:
# Fit data to Grid Search
search.fit(X_train,y_train);

In [78]:
# Best score 1
search.best_score_

0.45975222736782717

# Car_10

## Splitting of the dataset

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_10, test_size=0.3, random_state=0)

## Pipeline and GridSearch

In [25]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [26]:
num_transformer= MinMaxScaler()

In [27]:
cat_features=['target_status', 'shares_at_announcement', 'shares_acquired', 
              'consideration_offered',
               'cross_border', 'relatedness',
               'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
               'business_sector_target', 'cluster_category']

In [28]:
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['acquisition_count', 'rel_deal_value']),
    ('cat_transformer', cat_transformer, cat_features)], remainder='passthrough')

## Baseline

In [29]:
baseline_model_str = DummyClassifier(strategy="stratified") 

In [30]:
cross_val_score(baseline_model_str, X_train, y_train, cv=5, scoring='accuracy').mean()

0.3734149838971297

## SGDClassifier

In [38]:
model= SGDClassifier()

In [39]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)])

In [72]:
grid = {
    'model__alpha': [1e-2,0.05, 1e-1, 0.5, 1], 
    'model__loss': ['modified_huber'],
    'model__penalty': ['elasticnet'],
    'model__n_jobs': [-1],
    'model__fit_intercept': [True, False],
    'model__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'model__eta0':[0.000001],
    'model__max_iter':[1000],
    'model__n_iter_no_change': [3, 5, 8]
}

In [82]:
grid= {'model__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
       'model__penalty': ['l2', 'l1','elasticnet'],
       'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1],
       'model__l1_ratio': [0.15, 0.3, 0.5, 0.75],
       'model__fit_intercept':[True,False],
       'model__max_iter':[1000,10000],
       'model__tol':[0.001,0.01, 0.1],
       'model__shuffle':[True, False],
       'model__epsilon':[0.1,0.5, 0.7],
       'model__n_jobs':[-1], 
       'model__random_state':[1, 3, 5, None], 
       'model__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
       'model__eta0':[0.000001]}

In [83]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = 'accuracy',
                      cv = 5,
                      n_jobs=-1) 

In [ ]:
# Fit data to Grid Search
search.fit(X_train,y_train);

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

...

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
/Users/manonlaffly/.pyenv/ve

In [36]:
# Best score 1
search.best_score_

0.4781800230376831

In [37]:
search.best_params_

{'model__alpha': 0.1,
 'model__loss': 'modified_huber',
 'model__n_jobs': -1,
 'model__penalty': 'elasticnet'}

In [50]:
# Best score 2
search.best_score_

0.471876395777992

In [51]:
search.best_params_

{'model__alpha': 0.05,
 'model__eta0': 1e-06,
 'model__fit_intercept': False,
 'model__learning_rate': 'optimal',
 'model__loss': 'modified_huber',
 'model__n_jobs': -1,
 'model__penalty': 'elasticnet'}

In [55]:
# Best score 3
search.best_score_

0.47866663532288023

In [56]:
search.best_params_

{'model__alpha': 0.1,
 'model__eta0': 1e-06,
 'model__fit_intercept': True,
 'model__learning_rate': 'optimal',
 'model__loss': 'modified_huber',
 'model__n_jobs': -1,
 'model__penalty': 'elasticnet'}

In [60]:
# Best score 4
search.best_score_

0.48157925668210344

In [61]:
search.best_params_

{'model__alpha': 0.1,
 'model__eta0': 1e-06,
 'model__fit_intercept': False,
 'model__learning_rate': 'optimal',
 'model__loss': 'modified_huber',
 'model__n_jobs': -1,
 'model__penalty': 'elasticnet'}

In [65]:
# Best score 5
search.best_score_

0.476240626248854

In [66]:
search.best_params_

{'model__alpha': 0.1,
 'model__eta0': 1e-06,
 'model__fit_intercept': True,
 'model__learning_rate': 'optimal',
 'model__loss': 'modified_huber',
 'model__max_iter': 10000,
 'model__n_jobs': -1,
 'model__penalty': 'elasticnet'}

In [70]:
# Best score 6
search.best_score_

0.4786713368908531

In [71]:
search.best_params_

{'model__alpha': 0.1,
 'model__eta0': 1e-06,
 'model__fit_intercept': False,
 'model__learning_rate': 'optimal',
 'model__loss': 'modified_huber',
 'model__max_iter': 1000,
 'model__n_iter_no_change': 8,
 'model__n_jobs': -1,
 'model__penalty': 'elasticnet'}

In [75]:
# Best score 7
search.best_score_

0.48060603211170927

In [76]:
search.best_params_

{'model__alpha': 0.1,
 'model__eta0': 1e-06,
 'model__fit_intercept': False,
 'model__learning_rate': 'optimal',
 'model__loss': 'modified_huber',
 'model__max_iter': 1000,
 'model__n_iter_no_change': 8,
 'model__n_jobs': -1,
 'model__penalty': 'elasticnet'}

In [ ]:
# Best score 8
search.best_score_

In [ ]:
search.best_params_

# Interaction

## Target_status

In [49]:
dropdown_target_status = widgets.Dropdown(options = ['public', 'others'], description='Target status:')

In [50]:
dropdown_target_status

Dropdown(description='Target status:', options=('public', 'others'), value='public')

## Nbr of acquisitions:

In [65]:
dropdown_acquisition_count= widgets.IntText(
    value=7,
    description='Number of acquisition:',
    disabled=False
)

In [64]:
dropdown_acquisition_count

IntText(value=7, description='Number of acquisition:', disabled=True)

## shares_at_announcement

In [60]:
dropdown_shares_at_announcement= widgets.Dropdown(options = ['yes', 'no'], description='Shares at announcement:')

Dropdown(description='Shares at announcement:', options=('yes', 'no'), value='yes')

## shares_acquired

In [66]:
dropdown_shares_acquired= widgets.Dropdown(options = ['full', 'not_full'], description='Shares acquired:')

In [67]:
dropdown_shares_acquired

Dropdown(description='Shares acquired:', options=('full', 'not_full'), value='full')

## consideration_offered

In [68]:
dropdown_consideration_offered= widgets.Dropdown(options = ['Cash', 'Other'], description='Consideration offered:')

In [69]:
dropdown_consideration_offered

Dropdown(description='Consideration offered:', options=('Cash', 'Other'), value='Cash')

## bidder_count

In [72]:
dropdown_bidder_count= widgets.IntText(
    value=1,
    description='Number of bidders:',
    disabled=False
)

In [73]:
dropdown_bidder_count

IntText(value=1, description='Number of bidders:')

## rel_deal_value

In [74]:
dropdown_rel_deal_value= widgets.FloatText(
    value=0.564,
    description='Real deal value:',
    disabled=False
)

In [75]:
dropdown_rel_deal_value

FloatText(value=0.564, description='Real deal value:')

## cross_border

In [76]:
dropdown_cross_border= widgets.Dropdown(options = ['cross_border', 'national'], description='Cross border merger ?')

In [77]:
dropdown_cross_border

Dropdown(description='Cross border merger ?', options=('cross_border', 'national'), value='cross_border')

## relatedness

In [79]:
dropdown_relatedness=widgets.Dropdown(options = ['not_related', 'industry', 'industry_group', 'economic_sector',
       'business_sector'], description='Are both companies related ?')

In [80]:
dropdown_relatedness

Dropdown(description='Are both companies related ?', options=('not_related', 'industry', 'industry_group', 'ec…

## economic_sector_ac

In [121]:
dropdown_economic_sector_ac= widgets.Dropdown(options = ['Consumer Non-Cyclicals', 'Financials', 'Basic Materials',
       'Energy', 'Industrials', 'Real Estate', 'Technology', 'Healthcare',
       'Consumer Cyclicals', 'Utilities',
       'Academic & Educational Services'], description='Economic sector of the acquiring company: ?')

In [124]:
dropdown_economic_sector_ac

Dropdown(description='Economic sector of the acquiring company: ?', options=('Consumer Non-Cyclicals', 'Financ…

## business_sector_ac

In [111]:
def get_business(econ_sector):
    list_econ= ['Consumer Non-Cyclicals', 'Financials', 'Basic Materials',
       'Energy', 'Industrials', 'Real Estate', 'Technology', 'Healthcare',
       'Consumer Cyclicals', 'Utilities',
       'Academic & Educational Services']
    if econ_sector in list_econ:
        list_business=list(data[data.economic_sector_ac == econ_sector]['business_sector_ac'].unique())
    return list_business

In [150]:
list(data[data.economic_sector_ac == 'Financials']['business_sector_ac'].unique())

['Banking & Investment Services',
 'Insurance',
 'Investment Holding Companies',
 'Collective Investments']

In [129]:
dropdown_business_sector_ac= widgets.Dropdown(options = get_business(dropdown_economic_sector_ac.value), description='Business sector of the acquiring:')

In [130]:
dropdown_business_sector_ac

Dropdown(description='Business sector of the acquiring:', options=('Food & Drug Retailing', 'Personal & Househ…

## economic_sector_target

In [131]:
dropdown_economic_sector_target= widgets.Dropdown(options = ['Consumer Non-Cyclicals', 'Financials', 'Basic Materials',
       'Energy', 'Industrials', 'Real Estate', 'Technology', 'Healthcare',
       'Consumer Cyclicals', 'Utilities',
       'Academic & Educational Services'], description='Economic sector of the target company: ?')

## business_sector_target

In [148]:
dropdown_business_sector_target= widgets.Dropdown(options = get_business(dropdown_economic_sector_target.value), description='Business sector of the target:')

## cluster_category

In [135]:
dropdown_cluster_category= widgets.Dropdown(options = ['divestiture', 'asset_driven_op', 'private_companies',
       'bankruptcy', 'possible_problematic', 'Unknown'], description='Acquisition technique: ')

In [136]:
dropdown_cluster_category

Dropdown(description='Acquisition technique: ', options=('divestiture', 'asset_driven_op', 'private_companies'…

## a_fin_adv_count

In [140]:
select_a_fin_adv_count=widgets.SelectMultiple(
    options=['Barclays', 'Bank of America', 'Citi', 'Credit Suisse','Goldman Sachs',
        'JP Morgan', 'Lazard', 'Morgan Stanley', 'Rothschild', 'UBS'],
    description='Acquiring financial advisor:',
    disabled=False
)

In [142]:
select_a_fin_adv_count

SelectMultiple(description='Acquiring financial advisor:', index=(1, 2, 3), options=('Barclays', 'Bank of Amer…

## t_fin_adv_count:

In [144]:
select_t_fin_adv_count=widgets.SelectMultiple(
    options=['Barclays', 'Bank of America', 'Citi', 'Credit Suisse','Goldman Sachs',
        'JP Morgan', 'Lazard', 'Morgan Stanley', 'Rothschild', 'UBS'],
    description='Target financial advisor:',
    disabled=False
)

## Concatination

In [149]:
input_widgets = widgets.VBox(
[dropdown_target_status, dropdown_acquisition_count, dropdown_shares_at_announcement, dropdown_shares_acquired,dropdown_consideration_offered,
 dropdown_bidder_count,dropdown_rel_deal_value, dropdown_cross_border, dropdown_relatedness,
 dropdown_economic_sector_ac, dropdown_business_sector_ac, dropdown_economic_sector_target, dropdown_business_sector_target,
 dropdown_cluster_category,select_a_fin_adv_count, select_t_fin_adv_count ])
display(input_widgets)


In [ ]:
dict_pred= 

In [41]:
prediction = 48
btn = widgets.Button(description='Get your prediction')
display(btn)
def btn_eventhandler(obj):
    print(f"The predicted CAR is: {prediction}".format(obj.description))
btn.on_click(btn_eventhandler)

Button(description='Get your prediction', style=ButtonStyle())

The predicted CAR is: 48
